# SYCL Common Parallel Patterns

## Learning Objectives
- What are some common patterns that we should understand?
- How do the patterns relate to the capabilities of different devices?
- Which patterns are already provided as SYCL functions and libraries?
- How would the patterns be implemented using direct programming?

# Common Parallel Patterns and Why?
We need to recognize parallel patterns that have proven to be useful in the parallel programming space and apply the techniques that are time-tested to be the best solution.

For example, MapReduce framework is adopted for big data applications; its success stems largely from being based on two simple yet effective parallel patterns, map and reduce.

There are a number of common patterns in parallel programming that crop up occasionally, independent of the programming language that we’re using.

These patterns are versatile and can be employed at any level of parallelism (e.g., sub-groups, work-groups, full devices) and on any device (e.g., CPUs, GPUs, FPGAs).

However, certain properties of the patterns (such as their scalability) may affect their suitability for different devices. We may be able to improve performance by selecting a different pattern entirely based on the selected device.

### Motivation

This module helps us understand what some of the common patterns are, how the patterns relate to the capabilities of different devices, what patterns are already provided as SYCL functions and libraries, and how we can implement them using SYCL.

The module focuses on some of the algorithmic patterns that are most useful for writing data-parallel kernels to help SYCL programmers be more productive.

# Local Memory Usage

Often, work-items need to share data and communicate with each other. On one hand, all work-items in all work-groups can access global memory, so data sharing and communication can occur through global memory. However, sharing and communication through global memory is less efficient, due to its lower bandwidth and higher latency. On the other hand, work-items in a sub-group that execute simultaneously in an execution unit (EU) thread can share data and communicate with each other very efficiently; however, the number of work-items in a sub-group is usually small and the scope of data sharing and communication is very limited.

Memory with higher bandwidth and lower latency that’s accessible to a bigger scope of work-items is very desirable for data sharing communication among work-items. The shared local memory (SLM) in GPUs is designed for this purpose.

To simplify kernel development and accelerate communication between work-items in a work-group, SYCL defines a special local memory space specifically for communication between work-items in a work-group.

<img src="assets/localmem.png">

Each work-group may access variables in its own local memory space, but it cannot access variables in another work-group’s local memory. When a work-group begins , the contents of its local memory are uninitialized, and local memory does not persist after a work-group finishes executing. Because of these properties, local memory may only be used for temporary storage while a work-group is executing.

For some devices, such as for many CPU devices, local memory is a software abstraction and is implemented using the same memory subsystems as global memory. On these devices, using local memory is primarily a convenience mechanism for communication. Some compilers may use the memory space information for compiler local memory its own local memory optimizations, but otherwise using local memory for communication will not fundamentally perform better than communication via global memory on these devices.

For other devices though, such as many GPU devices, there are dedicated resources for local memory, and on these devices, communicating via local memory will perform better than communicating through global memory.

We can use the device query `info::device::local_mem_type` to determine whether an accelerator has dedicated resources for local memory or whether local memory is implemented as a software abstraction of global memory. 

We can use the device query `info::device::local_mem_size` to determine the size of local memory available for each work-group to access.

### Local Memory Type and Size

The code below uses device query to determine the local memory size and type. Inspect code, there are no modifications necessary:
1. Inspect the code cell below and click run ▶ to save the code to file.
2. Next run ▶ the cell in the __Build and Run__ section below the code to compile and execute the code.

In [ ]:
%%writefile lab/localmem_info.cpp
//==============================================================
// Copyright © Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================
#include <sycl/sycl.hpp>

using namespace sycl;

int main() {
  queue q;

  //# Print the device info
  std::cout << "device name   : " << q.get_device().get_info<info::device::name>() << "\n";
  std::cout << "local_mem_size: " << q.get_device().get_info<info::device::local_mem_size>() << "\n";

  auto local_mem_type = q.get_device().get_info<info::device::local_mem_type>();
  if(local_mem_type == info::local_mem_type::local) 
    std::cout << "local_mem_type: info::local_mem_type::local" << "\n";
  else if(local_mem_type == info::local_mem_type::global) 
    std::cout << "local_mem_type: info::local_mem_type::global" << "\n";
  else if(local_mem_type == info::local_mem_type::none) 
    std::cout << "local_mem_type: info::local_mem_type::none" << "\n";
 
  return 0;
}

#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [ ]:
! chmod 755 q; chmod 755 run_localmem_info.sh; if [ -x "$(command -v qsub)" ]; then ./q run_localmem_info.sh; else ./run_localmem_info.sh; fi

### Local Accessors

A local accessor is used to declare local memory for use in an ND-range kernel. Like other accessor objects, a local accessor is constructed within a command group handler.

A local accessor is created by specifying a type and a range describing the number of elements of that type. Like other accessors, local accessors may be one-dimensional, two-dimensional, or three-dimensional.

Below is an example of defining local_accessor `localmem` with type _int_ and _one-dimension_ data

```cpp
local_accessor<int, 1> localmem(N, h);
```

Below is an example of defining local_accessor `localmem` with type _float_ and _two-dimension_ data

```cpp
local_accessor<float, 2> localmem(range<2>(N, N), h);
```

The local accessor from one work-group can be accessed by all work-items within the work-group. Each work-group can have its own local accessor; a work-item from another work-group cannot access this local accessor.

### Group Barrier

When local accessor data is shared, work-group barriers are often required for work-item synchronization.

The `group_barrier` function synchronizes how each work-item views the state of memory. This type of synchronization operation is known as enforcing memory consistency or fencing memory. It ensures that the results of memory operations performed before the barrier are visible to other work-items after the
barrier.

A `group_barrier` is usually required right after a local accessor is modified by a work-item so that it is synchronized for all work-items before the local accessor can be accessed.

Below is an example of how a `group_barrier` function is defined to synchronize across all work-items within the work-group:

```cpp
group_barrier(item.get_group());
```

# Map
The map pattern is the simplest parallel pattern in functional programming languages.

* Each input element of a range is independently mapped to an output by applying some function.
* Many data-parallel operations can be expressed as instances of the map pattern (e.g., vector addition.)
* Since every application of the function is completely independent, expressions of map are often very simple, and the compiler and/ or runtime will do most of the hard work.
* Kernels written to the map pattern are suitable for any device and for the performance of those kernels to scale very well with the amount of available hardware parallellism.
* Such a development approach is highly productive and guarantees that an application will be portable to a wide variety of devices.

### Map example

The code below uses basic map implementation using SYCL. Inspect the code, there are no modifications necessary:

1. Inspect the code cell below and click run ▶ to save the code to file.
2. Next run ▶ the cell in the __Build and Run__ section below the code to compile and execute the code.

In [ ]:
%%writefile lab/basic_map.cpp
//==============================================================
// Copyright © Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================

#include <algorithm>
#include <cstdio>
#include <cstdlib>
#include <numeric>
#include <cmath>
#include <sycl/sycl.hpp>

using namespace sycl;

int main() {
  queue q;

  const size_t N = 64;
  float* input = malloc_shared<float>(N, q);
  float* output = malloc_shared<float>(N, q);
  std::iota(input, input + N, 1);
  std::fill(output, output + N, 0);

  // BEGIN CODE SNIP
  // Compute the square root of each input value
  q.parallel_for(N, [=](id<1> i) {
     output[i] = std::sqrt(input[i]);
   }).wait();
  // END CODE SNIP

  // Check that all outputs match serial execution.
  bool passed = true;
  for (int i = 0; i < N; ++i) {
    float gold = std::sqrt(input[i]);
    if (std::abs(output[i] - gold) >= 1.0E-06) {
      passed = false;
    }
  }
  std::cout << ((passed) ? "SUCCESS" : "FAILURE") << "\n";

  free(output, q);
  free(input, q);
  return (passed) ? 0 : 1;
}

#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [ ]:
! chmod 755 q; chmod 755 run_map.sh; if [ -x "$(command -v qsub)" ]; then ./q run_map.sh; else ./run_map.sh; fi

## Stencil
* The stencil pattern is closely related to the map pattern.
* A function is applied to an input and a set of neighboring inputs described by a stencil to produce a single output.
* Stencil patterns appear frequently in many domains, including scientific/engineering applications (e.g., finite difference codes) and computer vision/machine learning applications (e.g., image convolutions)..
* When the stencil pattern is executed out-of-place (i.e., writing the outputs to a separate storage location), the function can be applied to every input independently.
* Computing neighboring outputs requires the same data, and loading that data from memory multiple times will degrade performance;
* We may consider applying the stencil in-place (i.e., overwriting the original input values) in order to decrease an application’s memo footprint.
* The suitability of a stencil kernel for different devices is therefore highly dependent on the properties of the stencil and the input problem

    *  Small stencils can benefit from the scratchpad storage of GPUs.
    *  Large stencils can benefit from the (comparatively) large caches of CPUs.

### Stencil without Local Memory

The code below implements a stencil directly as a multidimensional basic data-parallel kernel with multidimensional buffers.
Inspect the code, there are no modifications necessary:

1. Inspect the code cell below and click run ▶ to save the code to file.
2. Next run ▶ the cell in the __Build and Run__ section below the code to compile and execute the code.

In [ ]:
%%writefile lab/stencil.cpp
//==============================================================
// Copyright © Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================


#include <algorithm>
#include <cstdio>
#include <cstdlib>
#include <iostream>
#include <numeric>
#include <random>
#include <sycl/sycl.hpp>

using namespace sycl;

int main() {
  queue q;

  const size_t N = 16;
  const size_t M = 16;
  range<2> stencil_range(N, M);
  range<2> alloc_range(N + 2, M + 2);
  std::vector<float> input(alloc_range.size()),
      output(alloc_range.size());
  std::iota(input.begin(), input.end(), 1);
  std::fill(output.begin(), output.end(), 0);

  {
    buffer<float, 2> input_buf(input.data(), alloc_range);
    buffer<float, 2> output_buf(output.data(), alloc_range);

    // BEGIN CODE SNIP
    q.submit([&](handler& h) {
      accessor input{input_buf, h};
      accessor output{output_buf, h};

      // Compute the average of each cell and its immediate
      // neighbors
      h.parallel_for(stencil_range, [=](id<2> idx) {
        int i = idx[0] + 1;
        int j = idx[1] + 1;

        float self = input[i][j];
        float north = input[i - 1][j];
        float east = input[i][j + 1];
        float south = input[i + 1][j];
        float west = input[i][j - 1];
        output[i][j] =
            (self + north + east + south + west) / 5.0f;
      });
    });
    // END CODE SNIP
  }

  // Check that all outputs match serial execution.
  bool passed = true;
  for (int i = 1; i < N + 1; ++i) {
    for (int j = 1; j < M + 1; ++j) {
      float self = input[i * (M + 2) + j];
      float north = input[(i - 1) * (M + 2) + j];
      float east = input[i * (M + 2) + (j + 1)];
      float south = input[(i + 1) * (M + 2) + j];
      float west = input[i * (M + 2) + (j - 1)];
      float gold =
          (self + north + east + south + west) / 5.0f;
      if (std::abs(output[i * (M + 2) + j] - gold) >=
          1.0E-06) {
        passed = false;
      }
    }
  }
  std::cout << ((passed) ? "SUCCESS" : "FAILURE") << "\n";
  return (passed) ? 0 : 1;
}



#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [ ]:
! chmod 755 q; chmod 755 run_stencil.sh; if [ -x "$(command -v qsub)" ]; then ./q run_stencil.sh; else ./run_stencil.sh; fi

### Stencil with Local Memory

The above example of the stencil pattern is very naïve and should not be expected to perform very well.

Leveraging locality (via spatial or temporal blocking) is required to avoid repeated reads of the same data from memory. 
A simple example of spatial blocking, using work-group local memory, is shown in the code below. 
Inspect the code, there are no modifications necessary:

1. Inspect the code cell below and click run ▶ to save the code to file.
2. Next run ▶ the cell in the __Build and Run__ section below the code to compile and execute the code.

In [ ]:
%%writefile lab/stencil_localmem.cpp
//==============================================================
// Copyright © Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================

#include <algorithm>
#include <cstdio>
#include <cstdlib>
#include <iostream>
#include <numeric>
#include <random>
#include <sycl/sycl.hpp>

using namespace sycl;

int main() {
  queue q;

  const size_t N = 16;
  const size_t M = 16;
  range<2> stencil_range(N, M);
  range<2> alloc_range(N + 2, M + 2);
  std::vector<float> input(alloc_range.size()),
      output(alloc_range.size());
  std::iota(input.begin(), input.end(), 1);
  std::fill(output.begin(), output.end(), 0);

  {
    // Create SYCL buffers associated with input/output
    buffer<float, 2> input_buf(input.data(), alloc_range);
    buffer<float, 2> output_buf(output.data(), alloc_range);

    // BEGIN CODE SNIP
    q.submit([&](handler& h) {
      accessor input{input_buf, h};
      accessor output{output_buf, h};

      constexpr size_t B = 4;
      range<2> local_range(B, B);
      range<2> tile_size =
          local_range +
          range<2>(2, 2);  // Includes boundary cells
      auto tile = local_accessor<float, 2>(tile_size, h);

      // Compute the average of each cell and its immediate
      // neighbors
      h.parallel_for(
          nd_range<2>(stencil_range, local_range),
          [=](nd_item<2> it) {
            // Load this tile into work-group local memory
            id<2> lid = it.get_local_id();
            range<2> lrange = it.get_local_range();
            for (int ti = lid[0]; ti < B + 2;
                 ti += lrange[0]) {
              int gi = ti + B * it.get_group(0);
              for (int tj = lid[1]; tj < B + 2;
                   tj += lrange[1]) {
                int gj = tj + B * it.get_group(1);
                tile[ti][tj] = input[gi][gj];
              }
            }
            group_barrier(it.get_group());

            // Compute the stencil using values from local
            // memory
            int gi = it.get_global_id(0) + 1;
            int gj = it.get_global_id(1) + 1;

            int ti = it.get_local_id(0) + 1;
            int tj = it.get_local_id(1) + 1;

            float self = tile[ti][tj];
            float north = tile[ti - 1][tj];
            float east = tile[ti][tj + 1];
            float south = tile[ti + 1][tj];
            float west = tile[ti][tj - 1];
            output[gi][gj] =
                (self + north + east + south + west) / 5.0f;
          });
    });
    // END CODE SNIP
  }

  // Check that all outputs match serial execution
  bool passed = true;
  for (int i = 1; i < N + 1; ++i) {
    for (int j = 1; j < M + 1; ++j) {
      float self = input[i * (M + 2) + j];
      float north = input[(i - 1) * (M + 2) + j];
      float east = input[i * (M + 2) + (j + 1)];
      float south = input[(i + 1) * (M + 2) + j];
      float west = input[i * (M + 2) + (j - 1)];
      float gold =
          (self + north + east + south + west) / 5.0f;
      if (std::abs(output[i * (M + 2) + j] - gold) >=
          1.0E-06) {
        passed = false;
      }
    }
  }
  std::cout << ((passed) ? "SUCCESS" : "FAILURE") << "\n";
  return (passed) ? 0 : 1;
}



#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [ ]:
! chmod 755 q; chmod 755 run_stencil_local.sh; if [ -x "$(command -v qsub)" ]; then ./q run_stencil_local.sh; else ./run_stencil_local.sh; fi

### ISO2DFD 

ISO2DFS is a wave propagation kernel used in Oil and Gas applications. The resolution of the wave equation is based on finite differences which results in implementing a stencil in a 2D volume.

ISO2DFD is a finite difference stencil kernel for solving the 2D acoustic isotropic wave equation. Kernels in this sample are implemented as 2nd order in space, 2nd order in time scheme without boundary conditions. Using SYCL the sample will explicitly run on the GPU as well as CPU to calculate a result.

Inspect the code, there are no modifications necessary:

1. Inspect the code cell below and click run ▶ to save the code to file.
2. Next run ▶ the cell in the __Build and Run__ section below the code to compile and execute the code.



In [ ]:
%%writefile lab/iso2dfd.cpp
//==============================================================
// Copyright © Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================

// ISO2DFD: 2D-Finite-Difference-Wave
// Propagation
//
// A complete online tutorial for this code sample can be found at :
// https://software.intel.com/en-us/articles/code-sample-two-dimensional-finite-difference-wave-propagation-in-isotropic-media-iso2dfd


#include <fstream>
#include <iostream>
#include <string>
#include <sycl/sycl.hpp>
#include <cmath>
#include <cstring>
#include <stdio.h>

#include "dpc_common.hpp"

using namespace sycl;
using namespace std;

/*
 * Parameters to define coefficients
 * half_length: Radius of the stencil
 * Sample source code is tested for half_length=1 resulting in
 * 2nd order Stencil finite difference kernel
 */

constexpr float DT = 0.002f;
constexpr float DXY = 20.0f;
constexpr unsigned int half_length = 1;

/*
 * Host-Code
 * Utility function to display input arguments
 */
void Usage(const string &program_name) {
  cout << " Incorrect parameters\n";
  cout << " Usage: ";
  cout << program_name << " n1 n2 Iterations\n\n";
  cout << " n1 n2      : Grid sizes for the stencil\n";
  cout << " Iterations : No. of timesteps.\n";
}

/*
 * Host-Code
 * Function used for initialization
 */
void Initialize(float* ptr_prev, float* ptr_next, float* ptr_vel, size_t n_rows,
                size_t n_cols) {
  cout << "Initializing ...\n";

  // Define source wavelet
  float wavelet[12] = {0.016387336, -0.041464937, -0.067372555, 0.386110067,
                       0.812723635, 0.416998396,  0.076488599,  -0.059434419,
                       0.023680172, 0.005611435,  0.001823209,  -0.000720549};

  // Initialize arrays
  for (size_t i = 0; i < n_rows; i++) {
    size_t offset = i * n_cols;

    for (int k = 0; k < n_cols; k++) {
      ptr_prev[offset + k] = 0.0f;
      ptr_next[offset + k] = 0.0f;
      // pre-compute squared value of sample wave velocity v*v (v = 1500 m/s)
      ptr_vel[offset + k] = (1500.0f * 1500.0f);
    }
  }
  // Add a source to initial wavefield as an initial condition
  for (int s = 11; s >= 0; s--) {
    for (int i = n_rows / 2 - s; i < n_rows / 2 + s; i++) {
      size_t offset = i * n_cols;
      for (int k = n_cols / 2 - s; k < n_cols / 2 + s; k++) {
        ptr_prev[offset + k] = wavelet[s];
      }
    }
  }
}

/*
 * Host-Code
 * Utility function to print device info
 */
void PrintTargetInfo(queue& q) {
  auto device = q.get_device();
  auto max_block_size =
      device.get_info<info::device::max_work_group_size>();

  auto max_EU_count =
      device.get_info<info::device::max_compute_units>();

  cout<< " Running on " << device.get_info<info::device::name>()<<"\n";
  cout<< " The Device Max Work Group Size is : "<< max_block_size<<"\n";
  cout<< " The Device Max EUCount is : " << max_EU_count<<"\n";
}

/*
 * Host-Code
 * Utility function to calculate L2-norm between resulting buffer and reference
 * buffer
 */
bool WithinEpsilon(float* output, float* reference, const size_t dim_x,
                    const size_t dim_y, const unsigned int radius,
                    const float delta = 0.01f) {
  ofstream err_file;
  err_file.open("error_diff.txt");

  bool error = false;
  double norm2 = 0;

  for (size_t iy = 0; iy < dim_y; iy++) {
    for (size_t ix = 0; ix < dim_x; ix++) {
      if (ix >= radius && ix < (dim_x - radius) && iy >= radius &&
          iy < (dim_y - radius)) {
        float difference = fabsf(*reference - *output);
        norm2 += difference * difference;
        if (difference > delta) {
          error = true;
          err_file<<" ERROR: "<<ix<<", "<<iy<<"   "<<*output<<"   instead of "<<
                   *reference<<"  (|e|="<<difference<<")\n";
        }
      }

      ++output;
      ++reference;
    }
  }

  err_file.close();
  norm2 = std::sqrt(norm2);
  if (error) printf("error (Euclidean norm): %.9e\n", norm2);
  return error;
}

/*
 * Host-Code
 * CPU implementation for wavefield modeling
 * Updates wavefield for the number of iterations given in nIteratons parameter
 */
void Iso2dfdIterationCpu(float* next, float* prev, float* vel,
                            const float dtDIVdxy, int n_rows, int n_cols,
                            int n_iterations) {
  float* swap;
  float value = 0.0;
  int   gid = 0;
  for (unsigned int k = 0; k < n_iterations; k += 1) {
    for (unsigned int i = 1; i < n_rows - half_length; i += 1) {
      for (unsigned int j = 1; j < n_cols - half_length; j += 1) {
        value = 0.0;

        // Stencil code to update grid
        gid = j + (i * n_cols);
        value = 0.0;
        value += prev[gid + 1] - 2.0f * prev[gid] + prev[gid - 1];
        value += prev[gid + n_cols] - 2.0f * prev[gid] + prev[gid - n_cols];
        value *= dtDIVdxy * vel[gid];
        next[gid] = 2.0f * prev[gid] - next[gid] + value;
      }
    }

    // Swap arrays
    swap = next;
    next = prev;
    prev = swap;
  }
}

/*
 * Device-Code - GPU
 * SYCL implementation for single iteration of iso2dfd kernel
 *
 * Range kernel is used to spawn work-items in x, y dimension
 *
 */
void Iso2dfdIterationGlobal(id<2> it, float* next, float* prev,
                               float* vel, const float dtDIVdxy, int n_rows,
                               int n_cols) {
  float value = 0.0;

  // Compute global id
  // We can use the get.global.id() function of the item variable
  //   to compute global id. The 2D array is laid out in memory in row major
  //   order.
  size_t gid_row = it.get(0);
  size_t gid_col = it.get(1);
  size_t gid = (gid_row)*n_cols + gid_col;

  // Computation to solve wave equation in 2D
  // First check if gid is inside the effective grid (not in halo)
  if ((gid_col >= half_length && gid_col < n_cols - half_length) &&
      (gid_row >= half_length && gid_row < n_rows - half_length)) {
    // Stencil code to update grid point at position given by global id (gid)
    // New time step for grid point is computed based on the values of the
    //    the immediate neighbors in both the horizontal and vertical
    //    directions, as well as the value of grid point at a previous time step
    value = 0.0;
    value += prev[gid + 1] - 2.0f * prev[gid] + prev[gid - 1];
    value += prev[gid + n_cols] - 2.0f * prev[gid] + prev[gid - n_cols];
    value *= dtDIVdxy * vel[gid];
    next[gid] = 2.0f * prev[gid] - next[gid] + value;
  }
}

int main(int argc, char* argv[]) {
  // Arrays used to update the wavefield
  float* prev_base;
  float* next_base;
  float* next_cpu;
  // Array to store wave velocity
  float* vel_base;

  bool error = false;

  size_t n_rows, n_cols;
  unsigned int n_iterations;

  // Read parameters
  try {
    n_rows = stoi(argv[1]);
    n_cols = stoi(argv[2]);
    n_iterations = stoi(argv[3]);
  }

  catch (...) {
    Usage(argv[0]);
    return 1;
  }

  // Compute the total size of grid
  size_t n_size = n_rows * n_cols;

  // Allocate arrays to hold wavefield and velocity
  prev_base = new float[n_size];
  next_base = new float[n_size];
  next_cpu = new float[n_size];
  vel_base = new float[n_size];

  // Compute constant value (delta t)^2 (delta x)^2. To be used in wavefield
  // update
  float dtDIVdxy = (DT * DT) / (DXY * DXY);

  // Initialize arrays and introduce initial conditions (source)
  Initialize(prev_base, next_base, vel_base, n_rows, n_cols);

  cout << "Grid Sizes: " << n_rows << " " << n_cols << "\n";
  cout << "Iterations: " << n_iterations << "\n\n";

  // Create a device queue using SYCL class queue
  queue q(default_selector_v);

  cout << "Computing wavefield in device ..\n";
  // Display info about device
  PrintTargetInfo(q);

  // Start timer
  dpc_common::TimeInterval t_offload;

  {  // Begin buffer scope
    // Create buffers using SYCL class buffer
    buffer next_buf(next_base, range(n_size));
    buffer prev_buf(prev_base, range(n_size));
    buffer vel_buf(vel_base, range(n_size));

    // Iterate over time steps
    for (unsigned int k = 0; k < n_iterations; k += 1) {
      // Submit command group for execution
      q.submit([&](auto &h) {
        // Create accessors
        accessor next_a(next_buf, h);
        accessor prev_a(prev_buf, h);
        accessor vel_a(vel_buf, h, read_only);

        // Define local and global range
        auto global_range = range<2>(n_rows, n_cols);

        // Send a SYCL kernel (lambda) for parallel execution
        // The function that executes a single iteration is called
        // "iso_2dfd_iteration_global"
        //    alternating the 'next' and 'prev' parameters which effectively
        //    swaps their content at every iteration.
        if (k % 2 == 0)
          h.parallel_for(global_range, [=](auto it) {
                Iso2dfdIterationGlobal(it, const_cast<float*>(next_a.template get_multi_ptr<sycl::access::decorated::no>().get()),
                                          const_cast<float*>(prev_a.template get_multi_ptr<sycl::access::decorated::no>().get()), 
                                          const_cast<float*>(vel_a.template get_multi_ptr<sycl::access::decorated::no>().get()),
                                          dtDIVdxy, n_rows, n_cols);
              });
        else
          h.parallel_for(global_range, [=](auto it) {
                Iso2dfdIterationGlobal(it, const_cast<float*>(prev_a.template get_multi_ptr<sycl::access::decorated::no>().get()),
                                          const_cast<float*>(next_a.template get_multi_ptr<sycl::access::decorated::no>().get()), 
                                          const_cast<float*>(vel_a.template get_multi_ptr<sycl::access::decorated::no>().get()),
                                          dtDIVdxy, n_rows, n_cols);
              });
      });

    }  // end for

  }  // buffer scope

  // Wait for commands to complete. Enforce synchronization on the command queue
  q.wait_and_throw();

  // Compute and display time used by device
  auto time = t_offload.Elapsed();

  cout << "Offload time: " << time << " s\n\n";

  // Output final wavefield (computed by device) to binary file
  ofstream out_file;
  out_file.open("wavefield_snapshot.bin", ios::out | ios::binary);
  out_file.write(reinterpret_cast<char*>(next_base), n_size * sizeof(float));
  out_file.close();

  // Compute wavefield on CPU (for validation)
  
  cout << "Computing wavefield in CPU ..\n";
  // Re-initialize arrays
  Initialize(prev_base, next_cpu, vel_base, n_rows, n_cols);

  // Compute wavefield on CPU
  // Start timer for CPU
  dpc_common::TimeInterval t_cpu;

  Iso2dfdIterationCpu(next_cpu, prev_base, vel_base, dtDIVdxy, n_rows, n_cols,
                         n_iterations);

  // Compute and display time used by CPU
  time = t_cpu.Elapsed();

  cout << "CPU time: " << time << " s\n\n";

  // Compute error (difference between final wavefields computed in device and
  // CPU)
  error = WithinEpsilon(next_base, next_cpu, n_rows, n_cols, half_length, 0.1f);

  // If error greater than threshold (last parameter in error function), report
  if (error)
    cout << "Final wavefields from device and CPU are different: Error\n";
  else
    cout << "Final wavefields from device and CPU are equivalent: Success\n";

  // Output final wavefield (computed by CPU) to binary file
  out_file.open("wavefield_snapshot_cpu.bin", ios::out | ios::binary);
  out_file.write(reinterpret_cast<char*>(next_cpu), n_size * sizeof(float));
  out_file.close();

  cout << "Final wavefields (from device and CPU) written to disk\n";
  cout << "Finished.\n";

  // Cleanup
  delete[] prev_base;
  delete[] next_base;
  delete[] vel_base;

  return error ? 1 : 0;
}

#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [ ]:
! chmod 755 q; chmod 755 run_iso2dfd.sh; if [ -x "$(command -v qsub)" ]; then ./q run_iso2dfd.sh; else ./run_iso2dfd.sh; fi

## SCAN

* The scan pattern computes a generalized prefix sum using a binary associative operator, and each element of the output represents a partial result.
* A scan is inclusive if the partial sum for element i is the sum of all elements in the range [0, i] (i.e., the sum including i).
*  A scan is exclusive if the partial sum for element i is the sum of all elements in the range [0, i) (i.e., the sum excluding i).
* Scan has less opportunities for parallelism than other patterns but it is still possible to implement a parallel scan using multiple sweeps over the same data.
* The best device on which to execute a scan is highly dependent on problem size:
    *  Smaller problems are a better fit for a CPU.
    *  Larger problems will contain enough data parallelism to saturate a GPU.


### Prefix Sum (Inclusive Scan):

In this implementation, a random sequence of 2**n elements is given as input (n is a positive number). The algorithm computes the prefix sum in parallel. The result sequence is in ascending order.
Given a randomized sequence of numbers x0, x1, x2, ..., xn, this algorithm computes and returns a new sequence y0, y1, y2, ..., yn so that

```cpp
y0 = x0
y1 = x0 + x1
y2 = x0 + x1 + x2
.....
yn = x0 + x1 + x2 + ... + xn{i}}


1. Inspect the code cell below and click run ▶ to save the code to file.
2. Next run ▶ the cell in the __Build and Run__ section below the code to compile and execute the code.


In [ ]:
%%writefile lab/prefix_sum.cpp
//==============================================================
// Copyright © Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================


// PrefixSum: this code sample implements the inclusive scan (prefix sum) in
// parallel. That is, given a randomized sequence of numbers x0, x1, x2, ...,
// xn, this algorithm computes and returns a new sequence y0, y1, y2, ..., yn so
// that
//
// y0 = x0
// y1 = x0 + x1
// y2 = x0 + x1 + x2
// .....
// yn = x0 + x1 + x2 + ... + xn
//
// Below is the pseudo code for computing prefix sum in parallel:
//
// n is power of 2 (1, 2, 4 , 8, 16, ...):
//
// for i from 0 to  [log2 n] - 1 do
//   for j from 0 to (n-1) do in parallel
//     if j<2^i then
//       x_{j}^{i+1} <- x_{j}^{i}}
//     else
//       x_{j}^{i+1} <- x_{j}^{i} + x_{j-2^{i}}^{i}}
//
// In the above, the notation x_{j}^{i} means the value of the jth element of
// array x in timestep i. Given n processors to perform each iteration of the
// inner loop in constant time, the algorithm as a whole runs in O(log n) time,
// the number of iterations of the outer loop.
//

#include <iostream>
#include <string>
#include <cmath>
// dpc_common.hpp can be found in the dev-utilities include folder.
// e.g., $ONEAPI_ROOT/dev-utilities/<version>/include/dpc_common.hpp
#include "dpc_common.hpp"

using namespace sycl;
using namespace std;

void Show(int a[], int arraysize) {
  for (int i = 0; i < arraysize; ++i) {
    cout << a[i] << " ";
    if ((i % 16) == 15) cout << "\n";
  }

  cout << "\n";
  return;
}

int* ParallelPrefixSum(int* current, int* next, unsigned int nb, queue& q) {
  unsigned int two_power = 1;
  unsigned int num_iter = std::log2(nb);
  // unsigned int uintmax = UINT_MAX;
  int* result = NULL;

  //  cout << "uintmax " << uintmax << " " << log2(uintmax) << "\n";
  // Buffer scope
  {
    buffer sequence_buf(current, range(nb));
    buffer sequence_next_buf(next, range(nb));

    // Iterate over the necessary iterations.
    for (unsigned int iter = 0; iter < num_iter; iter++, two_power *= 2) {
      // Submit command group for execution
      q.submit([&](auto& h) {
        // Create accessors
        accessor sequence(sequence_buf, h);
        accessor sequence_next(sequence_next_buf, h);

        if (iter % 2 == 0) {
          h.parallel_for(nb, [=](id<1> j) {
            if (j < two_power) {
              sequence_next[j] = sequence[j];
            } else {
              sequence_next[j] = sequence[j] + sequence[j - two_power];
            }
          });  // end parallel for loop in kernel
          result = next;
        } else {
          h.parallel_for(nb, [=](id<1> j) {
            if (j < two_power) {
              sequence[j] = sequence_next[j];
            } else {
              sequence[j] = sequence_next[j] + sequence_next[j - two_power];
            }
          });  // end parallel for loop in kernel
          result = current;
        }
      });  // end device queue
    }      // end iteration
  }        // Buffer scope

  // Wait for commands to complete. Enforce synchronization on the command queue
  q.wait_and_throw();

  return result;
}
/*
void PrefixSum(int* x, unsigned int nb)
{
  unsigned int two_power = 1;
  unsigned int num_iter = std::log2(nb);
  int temp = 0;

  // Iterate over the necessary iterations
  for (unsigned int iter = 0; iter < num_iter; iter++, two_power*=2) {
    //Show(x, nb);
    //    cout << "two_power: " << two_power << "\n";
    for (unsigned int j = nb; j > 0; j--) {
      if (j < two_power) {
        x[j] = x[j];
      }
      else {
        x[j] = x[j] + x[j - two_power];
      }
    }
  }
}
*/
void Usage(string prog_name, int exponent) {
  cout << " Incorrect parameters\n";
  cout << " Usage: " << prog_name << " n k \n\n";
  cout << " n: Integer exponent presenting the size of the input array.\n";
  cout << "    The number of element in the array must be power of 2\n";
  cout << "    (e.g., 1, 2, 4, ...). Please enter the corresponding exponent\n";
  cout << "    betwwen 0 and " << exponent - 1 << ".\n";
  cout << " k: Seed used to generate a random sequence.\n";
}

int main(int argc, char* argv[]) {
  unsigned int nb, seed;
  int n, exp_max = std::log2(numeric_limits<int>::max());

  // Read parameters.
  try {
    n = stoi(argv[1]);

    // Verify the boundary of acceptance.
    if (n < 0 || n >= exp_max) {
      Usage(argv[0], exp_max);
      return -1;
    }

    seed = stoi(argv[2]);
    nb = pow(2, n);
  } catch (...) {
    Usage(argv[0], exp_max);
    return -1;
  }

  cout << "\nSequence size: " << nb << ", seed: " << seed;

  int num_iter = std::log2(nb);
  cout << "\nNum iteration: " << num_iter << "\n";

  // Create a device queue using SYCL class queue
  queue q(default_selector_v);

  cout << "Device: " << q.get_device().get_info<info::device::name>() << "\n";

  int* data = new int[nb];
  int* prefix_sum1 = new int[nb];
  int* prefix_sum2 = new int[nb];
  int* result = NULL;

  srand(seed);

  // Initialize data arrays
  for (int i = 0; i < nb; i++) {
    data[i] = prefix_sum1[i] = rand() % 10;
    prefix_sum2[i] = 0;
  }

  // Start timer
  dpc_common::TimeInterval t;

  result = ParallelPrefixSum(prefix_sum1, prefix_sum2, nb, q);

  auto elapsed_time = t.Elapsed();

  cout << "Elapsed time: " << elapsed_time << " s\n";

  //cout << "\ndata after transforming using parallel prefix sum result:";
  //Show(result, nb);

  bool equal = true;

  if (result[0] != data[0])
    equal = false;
  else {
    for (int i = 1; i < nb; i++) {
      if (result[i] != result[i - 1] + data[i]) {
        equal = false;
        break;
      }
    }
  }

  delete[] data;
  delete[] prefix_sum1;
  delete[] prefix_sum2;

  if (!equal) {
    cout << "\nFailed: " << std::endl;
    return -2;
  } else {
    cout << "\nSuccess!" << std::endl;
    return 0;
  }
}



#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [ ]:
! chmod 755 q; chmod 755 run_prefix_sum.sh; if [ -x "$(command -v qsub)" ]; then ./q run_prefix_sum.sh; else ./run_prefix_sum.sh; fi

### Scan in Parallel with Multiple Kernels

* As discussed before, implementing a parallel scan requires multiple sweeps over the data, with synchronization occurring between each sweep.

* Since SYCL does not provide a mechanism for synchronizing all work-items in an ND-range, a direct implementation of a device-wide scan must use multiple kernels that communicate partial results through global memory.

* The code below demonstrates an inclusive scan implemented using several kernels.
    * The first kernel distributes the input values across work-groups and, computes work-group local scans in work-group local memory
    * . The second kernel computes a local scan using a single work-group, this time over the final value from each block.
    * The third kernel combines these intermediate results to finalize the prefix sum. 

Inspect the code, there are no modifications necessary:
1. Inspect the code cell below and click run ▶ to save the code to file.
2. Next run ▶ the cell in the __Build and Run__ section below the code to compile and execute the code.


In [ ]:
%%writefile lab/basic_scan.cpp
//==============================================================
// Copyright © Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================

#include <algorithm>
#include <cstdio>
#include <cstdlib>
#include <iostream>
#include <numeric>
#include <random>
#include <sycl/sycl.hpp>

using namespace sycl;

int main() {
  queue q;

  const size_t N = 128;
  const size_t L = 16;
  const size_t G = N / L;

  int32_t* input = malloc_shared<int32_t>(N, q);
  int32_t* output = malloc_shared<int32_t>(N, q);
  std::iota(input, input + N, 1);
  std::fill(output, output + N, 0);

  // Create a temporary allocation that will only be used by
  // the device
  int32_t* tmp = malloc_device<int32_t>(G, q);

  // BEGIN CODE SNIP
  // Phase 1: Compute local scans over input blocks
  q.submit([&](handler& h) {
     auto local = local_accessor<int32_t, 1>(L, h);
     h.parallel_for(nd_range<1>(N, L), [=](nd_item<1> it) {
       int i = it.get_global_id(0);
       int li = it.get_local_id(0);

       // Copy input to local memory
       local[li] = input[i];
       group_barrier(it.get_group());

       // Perform inclusive scan in local memory
       for (int32_t d = 0; d <= log2((float)L) - 1; ++d) {
         uint32_t stride = (1 << d);
         int32_t update =
             (li >= stride) ? local[li - stride] : 0;
         group_barrier(it.get_group());
         local[li] += update;
         group_barrier(it.get_group());
       }

       // Write the result for each item to the output
       // buffer Write the last result from this block to
       // the temporary buffer
       output[i] = local[li];
       if (li == it.get_local_range()[0] - 1) {
         tmp[it.get_group(0)] = local[li];
       }
     });
   }).wait();
  // END CODE SNIP

  // BEGIN CODE SNIP
  // Phase 2: Compute scan over partial results
  q.submit([&](handler& h) {
     auto local = local_accessor<int32_t, 1>(G, h);
     h.parallel_for(nd_range<1>(G, G), [=](nd_item<1> it) {
       int i = it.get_global_id(0);
       int li = it.get_local_id(0);

       // Copy input to local memory
       local[li] = tmp[i];
       group_barrier(it.get_group());

       // Perform inclusive scan in local memory
       for (int32_t d = 0; d <= log2((float)G) - 1; ++d) {
         uint32_t stride = (1 << d);
         int32_t update =
             (li >= stride) ? local[li - stride] : 0;
         group_barrier(it.get_group());
         local[li] += update;
         group_barrier(it.get_group());
       }

       // Overwrite result from each work-item in the
       // temporary buffer
       tmp[i] = local[li];
     });
   }).wait();
  // END CODE SNIP

  // BEGIN CODE SNIP
  // Phase 3: Update local scans using partial results
  q.parallel_for(nd_range<1>(N, L), [=](nd_item<1> it) {
     int g = it.get_group(0);
     if (g > 0) {
       int i = it.get_global_id(0);
       output[i] += tmp[g - 1];
     }
   }).wait();
  // END CODE SNIP
  
  // Check that all outputs match serial execution
  bool passed = true;
  int32_t gold = 0;
  for (int i = 0; i < N; ++i) {
    gold += input[i];
    if (output[i] != gold) {
      passed = false;
    }
  }
  std::cout << ((passed) ? "SUCCESS" : "FAILURE") << "\n";

  free(tmp, q);
  free(output, q);
  free(input, q);
  return (passed) ? 0 : 1;
}


#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [ ]:
! chmod 755 q; chmod 755 run_scan.sh; if [ -x "$(command -v qsub)" ]; then ./q run_scan.sh; else ./run_scan.sh; fi

## Reductions

* A reduction is a common parallel pattern which combines partial results using an operator that is typically associative and commutative (e.g.,addition).

* The most common examples of reductions are computing a sum (e.g., while computing a dot product) or computing the minimum/ maximum value (e.g., using maximum velocity to set time-step size).

* Tuning a reduction for different devices is a delicate balancing act between the time spent computing partial results and the time spent combining them; 
using too little parallelism increases computation time, whereas using too much parallelism increases combination time.

### The SYCL Reduction Library

* Reductions can be expressed directly using builtin functionality of SYCL or vendor-provided libraries written in 
SYCL
*  Leveraging these functions and libraries is the best way to balanc performance, portability, and productivity in real large-scale softwar engineering projects

* SYCL provides a convenient abstraction for describing variables with reduction semantics. 

* This allowing implementations to select between different reduction algorithms for different combinations of device, data type, and reduction operation.


* The result of a reduction is not guaranteed to be written back to the original variable until the kernel has completed.
* Accessing the result of a reduction behaves identically to accessing any other variable in SYCL.
* Accessing a reduction result stored in a buffer requires the creation of an appropriate device or host accessor.
* Accessing a reduction result stored in a USM allocation may require explicit synchronization and/or memory movement.

* If a reduction is initialized using a buffer or a USM pointer, the reduction is a scalar reduction, operating on the first object in an array.
* If a reduction is initialized using a span, the reduction is an array reduction.
* Each component of an array reduction is independent and is equivalent to N scalar reductions with the same data type and operator.



#### Basic reduction object
The kernel in below code shows an example of using the reduction library. Note that the kernel body doesn’t contain any reference to reductions. All we must specify is that the kernel contains a reduction which combines instances of the sum variable using the plus functor. This provides enough information for an implementation to automatically generate an optimized reduction sequence.

The code below basic reductions. Inspect code, there are no modifications necessary:

1. Inspect the code cell below and click run ▶ to save the code to file.

2. Next run ▶ the cell in the __Build and Run__ section below the code to compile and execute the code.

In [ ]:
%%writefile lab/basic_reductions.cpp
//==============================================================
// Copyright © Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================


#include <iostream>
#include <numeric>
#include <sycl/sycl.hpp>

using namespace sycl;

int main() {
  constexpr size_t N = 16;

  queue q;
  int* data = malloc_shared<int>(N, q);
  int* sum = malloc_shared<int>(1, q);
  std::iota(data, data + N, 1);
  *sum = 0;

  q.submit([&](handler& h) {
     // BEGIN CODE SNIP
     h.parallel_for(
         range<1>{N}, reduction(sum, plus<>()),
         [=](id<1> i, auto& sum) { sum += data[i]; });
     // END CODE SNIP
   }).wait();

  std::cout << "sum = " << *sum << "\n";
  bool passed = (*sum == ((N * (N + 1)) / 2));
  std::cout << ((passed) ? "SUCCESS" : "FAILURE") << "\n";

  free(sum, q);
  free(data, q);
  return (passed) ? 0 : 1;
}


#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [ ]:
! chmod 755 q; chmod 755 run_basic_reductions.sh; if [ -x "$(command -v qsub)" ]; then ./q run_basic_reductions.sh; else ./run_basic_reductions.sh; fi

#### Array Reductions
* When working with array reductions, the reducer provides an additional subscript operator (i.e., operator[]), allowing access to individual elements of the array.
* Rather than returning a reference directly to an element of the array, this operator returns another reducer object,which exposes the same combine() function and shorthand operators as the reducers associated with a scalar reduction.
* Below code shows a simple example of a kernel using an array reduction to compute a histogram, where the subscript operator is used to access only the histogram bin that is updated by the work-item

The code below demonstrates histogram using array reductions. Inspect code, there are no modifications necessary:

1. Inspect the code cell below and click run ▶ to save the code to file.
2. Next run ▶ the cell in the __Build and Run__ section below the code to compile and execute the code.

In [ ]:
%%writefile lab/array_reductions.cpp
//==============================================================
// Copyright © Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================


#include <iostream>
#include <numeric>
#include <sycl/sycl.hpp>

using namespace sycl;

int main() {
  constexpr size_t N = 16;
  constexpr size_t B = 4;

  queue q;
  int* data = malloc_shared<int>(N, q);
  int* histogram = malloc_shared<int>(B, q);
  std::iota(data, data + N, 1);
  std::fill(histogram, histogram + B, 0);

  q.submit([&](handler& h) {
     // BEGIN CODE SNIP
     h.parallel_for(
         range{N},
         reduction(span<int, 16>(histogram, 16), plus<>()),
         [=](id<1> i, auto& histogram) {
           histogram[i % B]++;
         });
     // END CODE SNIP
   }).wait();

  bool passed = true;
  std::cout << "Histogram:" << std::endl;
  for (int b = 0; b < B; ++b) {
    std::cout << "bin[" << b << "]: " << histogram[b]
              << std::endl;
    passed &= (histogram[b] == N / B);
  }
  std::cout << ((passed) ? "SUCCESS" : "FAILURE") << "\n";

  free(histogram, q);
  free(data, q);
  return (passed) ? 0 : 1;
}


#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [ ]:
! chmod 755 q; chmod 755 run_array_reductions.sh; if [ -x "$(command -v qsub)" ]; then ./q run_array_reductions.sh; else ./run_array_reductions.sh; fi

### Histogram

The Histogram sample demonstrates a histogram that groups numbers together and provides the count of a particular number in the input. The code in this sample uses Intel® oneAPI DPC++ Library (oneDPL) APIs to offload the computation to selected devices.
se
This sample creates both dense and sparse histograms using oneDPL APIs, on an input array of 1,000 elements with values chosen randomly ranging from 0 to 9 (inclusive). To differentiate between sparse and dense histogram, the code ensures one of the values (number 4) never occurs in the input array. One bin will always equal 0.

For the dense histogram, all the bins (including the zero-size bins) are stred.
For the sparse algorithm, only non-zero sized bins are stored.

The code demonstratesw uses dense and sprase histogram. Inspect code, there are no modifications necesnels]_

1. Inspect the code cell below and click run ▶ to save the code tofile.

2. Next run ▶ the cell in the __Build and Run__ section below the code to compile and execute the code.

In [ ]:
%%writefile lab/histogram.cpp
//==============================================================
// Copyright © Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================
//==============================================================
// Copyright © 2020 Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================

// oneDPL headers should be included before standard headers
#include <oneapi/dpl/algorithm>
#include <oneapi/dpl/execution>
#include <oneapi/dpl/numeric>

#include <sycl/sycl.hpp>
#include <iostream>
#include <random>

// Dense algorithm stores all the bins, even if bin has 0 entries
// input array [4,4,1,0,1,2]
// output [(0,1) (1,2)(2,1)(3,0)(4,2)]
// On the other hand, the sparse algorithm excludes the zero-bin values
// i.e., for the sparse algorithm, the same input will give the following output
// [(0,1) (1,2)(2,1)(4,2)]

void dense_histogram(std::vector<uint64_t> &input) {
  const int N = input.size();
  sycl::buffer<uint64_t> histogram_buf{input.data(), sycl::range<1>(N)};

  // Combine the equal values together
  std::sort(oneapi::dpl::execution::dpcpp_default,
            oneapi::dpl::begin(histogram_buf), oneapi::dpl::end(histogram_buf));

  // num_bins is maximum value + 1
  int num_bins;
  {
    sycl::host_accessor histogram(histogram_buf, sycl::read_only);
    num_bins = histogram[N - 1] + 1;
  }
  sycl::buffer<uint64_t> histogram_new_buf{sycl::range<1>(num_bins)};
  auto val_begin = oneapi::dpl::counting_iterator<int>{0};

  // Determine the end of each bin of value
  oneapi::dpl::upper_bound(
      oneapi::dpl::execution::dpcpp_default, oneapi::dpl::begin(histogram_buf),
      oneapi::dpl::end(histogram_buf), val_begin, val_begin + num_bins,
      oneapi::dpl::begin(histogram_new_buf));

  // Compute histogram by calculating differences of cumulative histogram
  std::adjacent_difference(oneapi::dpl::execution::dpcpp_default,
                           oneapi::dpl::begin(histogram_new_buf),
                           oneapi::dpl::end(histogram_new_buf),
                           oneapi::dpl::begin(histogram_new_buf));

  std::cout << "success for Dense Histogram:\n";
  {
    sycl::host_accessor histogram_new(histogram_new_buf, sycl::read_only);
    std::cout << "[";
    for (int i = 0; i < num_bins; i++) {
      std::cout << "(" << i << ", " << histogram_new[i] << ") ";
    }
    std::cout << "]\n";
  }
}

void sparse_histogram(std::vector<uint64_t> &input) {
  const int N = input.size();
  sycl::buffer<uint64_t> histogram_buf{input.data(), sycl::range<1>(N)};

  // Combine the equal values together
  std::sort(oneapi::dpl::execution::dpcpp_default,
            oneapi::dpl::begin(histogram_buf), oneapi::dpl::end(histogram_buf));

  // Create new buffer to store the unique valuesand their count;
  // oneapi::dpl::reduce_by_segment requires a sufficient buffer size(for worst case).
  // TODO: To consider usage of just 'sort' and 'transform_reduce' for calculate
  // a final result.There is a guess that such approach is more effective.
  sycl::buffer<uint64_t> histogram_values_buf{sycl::range<1>(N)};
  sycl::buffer<uint64_t> histogram_counts_buf{sycl::range<1>(N)};

  sycl::buffer<uint64_t> _const_buf{sycl::range<1>(N)};
  std::fill(oneapi::dpl::execution::dpcpp_default,
            oneapi::dpl::begin(_const_buf), oneapi::dpl::end(_const_buf), 1);

  auto histogram_values_buf_begin = oneapi::dpl::begin(histogram_values_buf);
  auto histogram_counts_buf_begin = oneapi::dpl::begin(histogram_counts_buf);

  // Find the count of each value
  const auto result = oneapi::dpl::reduce_by_segment(
      oneapi::dpl::execution::dpcpp_default, oneapi::dpl::begin(histogram_buf),
      oneapi::dpl::end(histogram_buf), oneapi::dpl::begin(_const_buf),
      histogram_values_buf_begin,
      histogram_counts_buf_begin);

  const auto num_bins = result.first - histogram_values_buf_begin;
  assert(num_bins == result.second - histogram_counts_buf_begin);

  std::cout << "success for Sparse Histogram:\n";
  sycl::host_accessor histogram_value(histogram_values_buf, sycl::read_only);
  sycl::host_accessor histogram_count(histogram_counts_buf, sycl::read_only);
  std::cout << "[";
  for (int i = 0; i < num_bins; i++) {
      std::cout << "(" << histogram_value[i] << ", " << histogram_count[i]
                << ") ";
  }
  std::cout << "]\n";
}

int main(void) {
  const int N = 1000;
  std::vector<uint64_t> input;
  srand((unsigned)time(0));
  // initialize the input array with randomly generated values between 0 and 9
  for (int i = 0; i < N; i++) input.push_back(rand() % 9);

  // replacing all input entries of "4" with random number between 1 and 3
  // this is to ensure that we have at least one entry with zero-bin size,
  // which shows the difference between sparse and dense algorithm output
  for (int i = 0; i < N; i++)
    if (input[i] == 4) input[i] = rand() % 3;
  dense_histogram(input);
  sparse_histogram(input);
  return 0;
}


#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [ ]:
! chmod 755 q; chmod 755 run_histogram.sh; if [ -x "$(command -v qsub)" ]; then ./q run_histogram.sh; else ./run_histogram.sh; fi

## PACK and UNPACK

* Pack and unpack are also known as gather and scatter operations.
* These operations handle differences in how data is arranged in memory and howwe wish to present it to the compute resources.

### PACK
* The pack pattern,  discards elements of an input range based on a Boolean condition, packing the elements that are not discarded into contiguous locations of the output range.
* This Boolean condition could be a precomputed mask or could be computed online by applying some function to each input element.

* Since pack depends on an exclusive scan, implementing a pack that applies to all elements of an ND-range must also take place via global memory and over the course of several kernel enqueues.
* There is a common use case for pack that does not require the operation to be applied over all elements of an ND-range—namely, applying a pack only across items in a specific work-group or sub-group.

* The example shown is based 
on a real kernel from molecular dynamics simulations: the work-items  nthe sub-group assigned to particle i cooperate to identify all other particl swithin a fixed distance of i, and only the particles in this “neighbor list” wi lbe used to calculate the force acting on each particl

* Pack pattern never reorders elements .
* The elements that are packed into the output array appear in the same order as they did in the input.e.The code below demonstrates how such a pack operation could be used in a kernel to build a list of elements which require some additional postprocessing (in a future kernel). 
. Inspect code, there are no modifications necessary

1. Inspect the code cell below and click run ▶ to save the code to fiel

2. Next run ▶ the cell in the __Build and Run__ section below the code to compile and execute the code.

In [ ]:
%%writefile lab/local_pack.cpp
//==============================================================
// Copyright © Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================


#include <algorithm>
#include <cstdio>
#include <cstdlib>
#include <iostream>
#include <numeric>
#include <random>
#include <sycl/sycl.hpp>

using namespace sycl;

int main() {
  queue q;

  // Set parameters to control neighborhood size
  const float CUTOFF = 3.0f;
  const uint32_t MAX_K = 150;

  // Initialize input and output on the host
  const uint32_t Nx = 8, Ny = 8, Nz = 8;
  const uint32_t N = Nx * Ny * Nz;
  float3* position = malloc_shared<float3>(N, q);
  uint32_t* num_neighbors = malloc_shared<uint32_t>(N, q);
  uint32_t* neighbors =
      malloc_shared<uint32_t>(N * MAX_K, q);
  for (uint32_t x = 0; x < Nx; ++x) {
    for (uint32_t y = 0; y < Ny; ++y) {
      for (uint32_t z = 0; z < Nz; ++z) {
        position[z * Ny * Nx + y * Nx + x] = {x, y, z};
      }
    }
  }
  std::fill(num_neighbors, num_neighbors + N, 0);
  std::fill(neighbors, neighbors + N * MAX_K, 0);

  // BEGIN CODE SNIP
  range<2> global(N, 8);
  range<2> local(1, 8);
  q.parallel_for(nd_range<2>(global, local), [=](nd_item<2>
                                                     it) {
     int i = it.get_global_id(0);
     sub_group sg = it.get_sub_group();
     int sglid = sg.get_local_id()[0];
     int sgrange = sg.get_local_range()[0];

     uint32_t k = 0;
     for (int j = sglid; j < N; j += sgrange) {
       // Compute distance between i and neighbor j
       float r = distance(position[i], position[j]);

       // Pack neighbors that require
       // post-processing into a list
       uint32_t pack = (i != j) and (r <= CUTOFF);
       uint32_t offset =
           exclusive_scan_over_group(sg, pack, plus<>());
       if (pack) {
         neighbors[i * MAX_K + k + offset] = j;
       }

       // Keep track of how many neighbors have been
       // packed so far
       k += reduce_over_group(sg, pack, plus<>());
     }
     num_neighbors[i] =
         reduce_over_group(sg, k, maximum<>());
   }).wait();
  // END CODE SNIP

  // Check that all outputs match serial execution
  bool passed = true;
  for (int i = 0; i < N; ++i) {
    uint32_t k = 0;
    for (int j = 0; j < N; ++j) {
      float r = distance(position[i], position[j]);
      if (i != j and r <= CUTOFF) {
        if (neighbors[i * MAX_K + k] != j) {
          passed = false;
        }
        k++;
      }
    }
    if (num_neighbors[i] != k) {
      passed = false;
    }
  }
  std::cout << ((passed) ? "SUCCESS" : "FAILURE") << "\n";
  free(neighbors, q);
  free(num_neighbors, q);
  free(position, q);
  return (passed) ? 0 : 1;
}

#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [ ]:
! chmod 755 q; chmod 755 run_local_pack.sh; if [ -x "$(command -v qsub)" ]; then ./q run_local_pack.sh; else ./run_local_pack.sh; fi

### UNPACK

* The unpack pattern is the opposite of the pack pattern.
* Contiguous elements of an input range are unpacked into noncontiguous elements of an output range, leaving other elements untouched.
* The most obvious use case for this pattern is to unpack data that was previously packed, but it can also be used to fill in “gaps” in data resulting from some previous computation.

* Below code  demonstrates how such a sub-group unpack operation could be used to improve load balancing in a kernel with divergent control flow (in this case, computing the Mandelbrot set).* Each work-item is assigned a separate pixel to compute and iterates until convergence or a maximum number of iterations is reached. An unpack operation is then used to replace completed pixels with new pixels.

Inspect code, there are no modifications necessary:

1. Inspect the code cell below and click run ▶ to save the code to file.
2. Next run ▶ the cell in the __Build and Run__ section below the code to compile and execute the code.

In [ ]:
%%writefile lab/local_unpack.cpp
//==============================================================
// Copyright © Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================


#include <algorithm>
#include <complex>
#include <cstdio>
#include <cstdlib>
#include <fstream>
#include <iostream>
#include <numeric>
#include <random>
#include <sycl/sycl.hpp>

using namespace sycl;

const uint32_t max_iterations = 1024;
const uint32_t Nx = 1024, Ny = 768;

struct Parameters {
  float xc;
  float yc;
  float zoom;
  float zoom_px;
  float x_span;
  float y_span;
  float x0;
  float y0;
  float dx;
  float dy;
};

void reset(Parameters params, uint32_t i, uint32_t j,
           uint32_t& count, float& cr, float& ci, float& zr,
           float& zi) {
  count = 0;
  cr = params.x0 + i * params.dx;
  ci = params.y0 + j * params.dy;
  zr = zi = 0.0f;
}

bool next_iteration(Parameters params, uint32_t i,
                    uint32_t j, uint32_t& count, float& cr,
                    float& ci, float& zr, float& zi,
                    uint32_t* mandelbrot) {
  bool converged = false;
  if (i < Nx) {
    float next_zr = zr * zr - zi * zi;
    float next_zi = 2 * zr * zi;
    zr = next_zr + cr;
    zi = next_zi + ci;
    count++;

    // Mark that this value of i has converged
    // Output the i result for this value of i
    if (count >= max_iterations or
        zr * zr + zi * zi >= 4.0f) {
      converged = true;
      uint32_t px = j * Nx + i;
      mandelbrot[px] = count;
    }
  }
  return converged;
}

int main() {
  queue q;

  // Set up parameters to control divergence, image size,
  // etc.
  Parameters params;
  params.xc = 0.0f;
  params.yc = 0.0f;
  params.zoom = 1.0f;
  params.zoom_px = pow(2.0f, 3.0f - params.zoom) * 1e-3f;
  params.x_span = Nx * params.zoom_px;
  params.y_span = Ny * params.zoom_px;
  params.x0 = params.xc - params.x_span * 0.5f;
  params.y0 = params.yc - params.y_span * 0.5f;
  params.dx = params.zoom_px;
  params.dy = params.zoom_px;

  // Initialize output on the host
  uint32_t* mandelbrot =
      malloc_shared<uint32_t>(Ny * Nx, q);
  std::fill(mandelbrot, mandelbrot + Ny * Nx, 0);

  range<2> global(Ny, 8);
  range<2> local(1, 8);
  q.parallel_for(nd_range<2>(global, local), [=](nd_item<2>
                                                     it) {
     const uint32_t j = it.get_global_id(0);
     sub_group sg = it.get_sub_group();

     // Treat each row as a queue of i values to compute
     // Initially the head of the queue is at 0
     uint32_t iq = 0;

     // Initially each work-item in the sub-group works
     // on contiguous values
     uint32_t i = iq + sg.get_local_id()[0];
     iq += sg.get_local_range()[0];

     // Initialize the iterator variables
     uint32_t count;
     float cr, ci, zr, zi;
     if (i < Nx) {
       reset(params, i, j, count, cr, ci, zr, zi);
     }

     // BEGIN CODE SNIP
     // Keep iterating as long as one work-item has work to do
     while (any_of_group(sg, i < Nx)) {
       uint32_t converged = next_iteration(
           params, i, j, count, cr, ci, zr, zi, mandelbrot);
       if (any_of_group(sg, converged)) {
         // Replace pixels that have converged using an
         // unpack. Pixels that haven't converged are not
	 // replaced.
         uint32_t index = exclusive_scan_over_group(
             sg, converged, plus<>());
         i = (converged) ? iq + index : i;
         iq += reduce_over_group(sg, converged, plus<>());

         // Reset the iterator variables for the new i
         if (converged) {
           reset(params, i, j, count, cr, ci, zr, zi);
         }
       }
     }
    // END CODE SNIP
   }).wait();

  // Produce an image as a PPM file
  constexpr uint32_t max_color = 65535;
  std::ofstream ppm;
  ppm.open("mandelbrot.ppm");
  ppm << std::string("P6").c_str() << "\n";
  ppm << Nx << "\n";
  ppm << Ny << "\n";
  ppm << max_color << "\n";
  size_t eof = ppm.tellp();
  ppm.close();
  ppm.open("mandelbrot.ppm",
           std::ofstream::binary | std::ofstream::app);
  ppm.seekp(eof);
  std::vector<uint16_t> colors(Nx * Ny * 3);
  for (uint32_t px = 0; px < Nx * Ny; ++px) {
    const uint16_t color =
        (max_iterations - mandelbrot[px]) *
        (max_color / (double)max_iterations);
    colors[3 * px + 0] = color;
    colors[3 * px + 1] = color;
    colors[3 * px + 2] = color;
  }
  ppm.write((char*)colors.data(), 2 * colors.size());
  ppm.close();

  free(mandelbrot, q);
  return 0;
}

#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [ ]:
! chmod 755 q; chmod 755 run_local_pack.sh; if [ -x "$(command -v qsub)" ]; then ./q run_local_pack.sh; else ./run_local_pack.sh; fi

***
# Summary

In this module you learned:
* How to implement some of the most common parallel patterns using SYCL features,
* How to implement some of the most common parallel patterns using built-in functions and libraries.